In [ ]:
# Connect to the cluster and initiate spark session

from pyspark.sql import SparkSession

spark_session = SparkSession.builder\
.master("spark://192.168.2.70:7077") \
.appName("test_1")\
.config("spark.dynamicAllocation.enabled", True)\
.config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
.config("spark.shuffle.service.enabled", True)\
.config("spark.dynamicAllocation.executorIdleTimeout", "30s")\
.config("spark.cores.max", 4)\
.config("spark.ui.showConsoleProgress", False)\
.getOrCreate()

In [1]:
# Import black for code formatting

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# Load data and infer schema automatically from json keys

df_raw = spark_session.read.options(multiline=True, header=True, inferSchema=True).json(
    "sample_data.json"
)
df_raw.show(10)

In [ ]:
# Print to inspect schema

df_raw.printSchema()

In [2]:
# Step 1: Filter out columns that will not be used

cols_to_keep = ["author", "body", "created_utc", "score", "subreddit", "subreddit_id"]

df_reddit = df_raw.select([col for col in cols_to_keep])

NameError: name 'df_raw' is not defined

<IPython.core.display.Javascript object>

In [ ]:
# Step 2.1: Count the biggest subreddits (by number of posts)

df_subred_count = df_reddit.groupBy("subreddit_id").count()

In [ ]:
# Step 2.2: Get list with top 100 subreddit ids

import pandas as pd

df_count_pd = df_subred_count.toPandas()
df_top_100 = df_count_pd.sort_values(by="count", ascending=False).iloc[0:100]
top_100_subreds = df_top_100.columns
print(top_100_subreds)

In [ ]:
# Step 3.1: Find out who are active users

df_user_count = df_reddit.groupBy("author").count()

In [ ]:
# Step 3.2: Keep only users who have posted more than 10 times

